In [5]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
# Importing important libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt

In [2]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
train_ = pd.read_csv('/content/drive/MyDrive/train.csv')
test_ = pd.read_csv('/content/drive/MyDrive/test.csv')
test_labels = pd.read_csv('/content/drive/MyDrive/test_labels.csv')
#present_data = pd.read_csv('/content/drive/My Drive/toxic_comments_data/present_data.csv')

In [7]:
train_

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [8]:
train_['labels'] = train_[train_.columns[2:]].values.tolist()
train = train_[['comment_text', 'labels']]
train

,comment_text,labels
0,Explanation\nWhy the edits made under my usern...,"[0, 0, 0, 0, 0, 0]"
1,D'aww! He matches this background colour I'm s...,"[0, 0, 0, 0, 0, 0]"
2,"Hey man, I'm really not trying to edit war. It...","[0, 0, 0, 0, 0, 0]"
3,"""\nMore\nI can't make any real suggestions on ...","[0, 0, 0, 0, 0, 0]"
4,"You, sir, are my hero. Any chance you remember...","[0, 0, 0, 0, 0, 0]"
...,...,...
159566,""":::::And for the second time of asking, when ...","[0, 0, 0, 0, 0, 0]"
159567,You should be ashamed of yourself \n\nThat is ...,"[0, 0, 0, 0, 0, 0]"
159568,"Spitzer \n\nUmm, theres no actual article for ...","[0, 0, 0, 0, 0, 0]"
159569,And it looks like it was actually you who put ...,"[0, 0, 0, 0, 0, 0]"


In [9]:
test_

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [10]:
test_ = pd.merge(test_, test_labels, on='id')
test_

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",-1,-1,-1,-1,-1,-1


In [11]:
test_cleaned = test_[~(test_ == -1).any(axis=1)]
test_cleaned

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [12]:
test_cleaned['labels'] = test_cleaned[test_cleaned.columns[2:]].values.tolist()
test = test_cleaned[['comment_text', 'labels']]
test.to_csv('/content/drive/My Drive/output.csv', index=False)
test

<ipython-input-12-fbf1dee68bde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_cleaned['labels'] = test_cleaned[test_cleaned.columns[2:]].values.tolist()


,comment_text,labels
5,Thank you for understanding. I think very high...,"[0, 0, 0, 0, 0, 0]"
7,:Dear god this site is horrible.,"[0, 0, 0, 0, 0, 0]"
11,"""::: Somebody will invariably try to add Relig...","[0, 0, 0, 0, 0, 0]"
13,""" \n\n It says it right there that it IS a typ...","[0, 0, 0, 0, 0, 0]"
14,""" \n\n == Before adding a new product to the l...","[0, 0, 0, 0, 0, 0]"
...,...,...
153150,":Jerome, I see you never got around to this…! ...","[0, 0, 0, 0, 0, 0]"
153151,==Lucky bastard== \n http://wikimediafoundatio...,"[0, 0, 0, 0, 0, 0]"
153154,==shame on you all!!!== \n\n You want to speak...,"[0, 0, 0, 0, 0, 0]"
153155,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,"[1, 0, 1, 0, 1, 0]"


In [13]:
print(f"Train Dataset Shape: {train.shape}")
print(f"Validation Dataset Shape: {test.shape}")

Train Dataset Shape: (159571, 2)
Validation Dataset Shape: (63978, 2)


In [14]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
# Creating Dataset and DataLoader for neural net
class DetoxDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [16]:
train_set = DetoxDataset(train, tokenizer, MAX_LEN)
val_set = DetoxDataset(test, tokenizer, MAX_LEN)

In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
               }

val_params = {'batch_size': VALID_BATCH_SIZE,
              'shuffle': True,
              'num_workers': 0
             }

train_loader = DataLoader(train_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

In [16]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth"):
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict()
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved to {filename}")

In [21]:
# model = DetoxClass()
# model.to(device)

from transformers import BertForSequenceClassification, AdamW

# Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)  # We have 6 labels

model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [18]:
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss

# Optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Loss function
loss_function = BCEWithLogitsLoss()

model.train()
for epoch in range(EPOCHS):  # Number of epochs
    for _,data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids, labels=targets)
        optimizer.zero_grad()
        loss = outputs.loss

        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Loss:  {loss.item()}')
    save_checkpoint(model, optimizer, filename=f"/content/drive/MyDrive/checkpoint_epoch_{epoch+1}.pth")



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1, Loss:  0.04789979010820389
Checkpoint saved to /content/drive/MyDrive/checkpoint_epoch_1.pth


KeyboardInterrupt: 

In [22]:
checkpoint_path = "/content/drive/MyDrive/checkpoint_epoch_1.pth"  # Adjust the path and filename as necessary
checkpoint = torch.load(checkpoint_path,map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [23]:
class UnseenDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(inputs["token_type_ids"], dtype=torch.long)
        }

# Assuming df_unseen is your DataFrame containing unseen data
test = test.reset_index(drop=True)
unseen_set = UnseenDataset(test, tokenizer, MAX_LEN)
unseen_loader = DataLoader(unseen_set, batch_size=16, shuffle=False)  # Adjust batch size as needed


In [ ]:
def predict(model, data_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids)
            logits = outputs.logits
            probabilities = torch.sigmoid(logits).cpu().numpy()  # Convert logits to probabilities
            predictions.extend(probabilities)
    return predictions

# Get predictions for the unseen dataset
unseen_predictions = predict(model, unseen_loader)
print(unseen_predictions)


In [ ]:
unseen_predictions[1]

In [ ]:
len(unseen_predictions)

In [27]:
fin_output = test.copy()
fin_output

,comment_text,labels
0,Thank you for understanding. I think very high...,"[0, 0, 0, 0, 0, 0]"
1,:Dear god this site is horrible.,"[0, 0, 0, 0, 0, 0]"
2,"""::: Somebody will invariably try to add Relig...","[0, 0, 0, 0, 0, 0]"
3,""" \n\n It says it right there that it IS a typ...","[0, 0, 0, 0, 0, 0]"
4,""" \n\n == Before adding a new product to the l...","[0, 0, 0, 0, 0, 0]"
...,...,...
63973,":Jerome, I see you never got around to this…! ...","[0, 0, 0, 0, 0, 0]"
63974,==Lucky bastard== \n http://wikimediafoundatio...,"[0, 0, 0, 0, 0, 0]"
63975,==shame on you all!!!== \n\n You want to speak...,"[0, 0, 0, 0, 0, 0]"
63976,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,"[1, 0, 1, 0, 1, 0]"


In [28]:
fin_output['preds'] = unseen_predictions

In [30]:
fin_output.to_csv('/content/drive/MyDrive/final_output.csv')

In [32]:
final_predictions = [(probs >= 0.5).astype(int) for probs in unseen_predictions]
len(final_predictions)

63978

In [33]:
fin_output['fin_preds'] = final_predictions

In [34]:
fin_output

,comment_text,labels,preds,fin_preds
0,Thank you for understanding. I think very high...,"[0, 0, 0, 0, 0, 0]","[0.0015550547, 0.00054885575, 0.0005997905, 0....","[0, 0, 0, 0, 0, 0]"
1,:Dear god this site is horrible.,"[0, 0, 0, 0, 0, 0]","[0.24580474, 0.0008883732, 0.0072826697, 0.001...","[0, 0, 0, 0, 0, 0]"
2,"""::: Somebody will invariably try to add Relig...","[0, 0, 0, 0, 0, 0]","[0.084569015, 0.0007317209, 0.009127388, 0.000...","[0, 0, 0, 0, 0, 0]"
3,""" \n\n It says it right there that it IS a typ...","[0, 0, 0, 0, 0, 0]","[0.0014481305, 0.0005016262, 0.0006038342, 0.0...","[0, 0, 0, 0, 0, 0]"
4,""" \n\n == Before adding a new product to the l...","[0, 0, 0, 0, 0, 0]","[0.0012948642, 0.0005683998, 0.0005933127, 0.0...","[0, 0, 0, 0, 0, 0]"
...,...,...,...,...
63973,":Jerome, I see you never got around to this…! ...","[0, 0, 0, 0, 0, 0]","[0.0014101207, 0.0004935659, 0.00061053585, 0....","[0, 0, 0, 0, 0, 0]"
63974,==Lucky bastard== \n http://wikimediafoundatio...,"[0, 0, 0, 0, 0, 0]","[0.8812886, 0.01128686, 0.6079708, 0.002962233...","[1, 0, 1, 0, 0, 0]"
63975,==shame on you all!!!== \n\n You want to speak...,"[0, 0, 0, 0, 0, 0]","[0.6585876, 0.02843133, 0.081317574, 0.0332228...","[1, 0, 0, 0, 0, 1]"
63976,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,"[1, 0, 1, 0, 1, 0]","[0.98643017, 0.16983761, 0.9423132, 0.02045822...","[1, 0, 1, 0, 1, 0]"


In [35]:
fin_output.to_csv('/content/drive/MyDrive/final_final_test.csv')